****LA SEMILLA DE LA COMPOSITACIÓN****

Heber Hernández Guerra, 2018

En este ejemplo de compositación de sondaje, hablaremos de 3 matrices:
* La primera será llamada matriz de entrada [A] (datos originales).
* la segunda será llamada matriz de salida [B] (datos compositados).
* la tercera será llamada matriz de desarrollo (medio para converir [A] en [B]).

En cuanto al curso:

* Tanto el dataset N° 1 como el dataset N° 2 ya estan compositados.

En cuanto a librerias con la función de compositar:

* *Pygslib* es una librería de python que contiene una función para compositar. Dicha libreria la revisaremos en otra unidad.


In [27]:
import pandas as pd
import numpy as np
from numpy import random

d = {'dhid': [1,1,1,1,1], 'from': [0,1,3,8,15], 'to': [1,3,8,15,20]}

df = pd.DataFrame(data=d)


g = np.arange(5)  # [0, 1, 2, 3, 4]
random.shuffle(g)  # mezclar!

df['grade'] = g

#Podemos crear un DataFrame con leyes (grade) aleatoriamente dentro de un intervalo.

df.head()

,dhid,from,to,grade
0,1,0,1,3
1,1,1,3,2
2,1,3,8,4
3,1,8,15,0
4,1,15,20,1


In [28]:
d = {'dhid': [1,1,1,1,1], 'from': [0,1,3,8,15], 'to': [1,3,8,15,20], 'grade': [1,2,3,4,5]}

df = pd.DataFrame(data=d)

#Podemos tambien dejar estaticos los valores "grade".

df.head()

,dhid,from,to,grade
0,1,0,1,1
1,1,1,3,2
2,1,3,8,3
3,1,8,15,4
4,1,15,20,5


Normalmente las tablas "assay", vienen con dicho formato. **Agregaremos la longitud**: *len*

In [29]:
df['len'] = (df['to'] - df['from']) # len = from - to

df.head()

,dhid,from,to,grade,len
0,1,0,1,1,1
1,1,1,3,2,2
2,1,3,8,3,5
3,1,8,15,4,7
4,1,15,20,5,5


Tambien agregaremos una longitud acumulada: *len_acum*

In [30]:
df['len_acum'] = df.len.cumsum() #suma acumulada de la variable "len"

df.head() #df de entrada, se sugiere guardar pd.to_csv() para no variar las leyes (grades)

,dhid,from,to,grade,len,len_acum
0,1,0,1,1,1,1
1,1,1,3,2,2,3
2,1,3,8,3,5,8
3,1,8,15,4,7,15
4,1,15,20,5,5,20


**Conociendo de la librería Pygslib la creación de la matriz de compositos:**
    
    #create the composite arrays
    
* cfrom = np.arange(0.,ito[-1]+cint,cint)     "cfrom" es un rango que parte en "0" hasta ultima posición de "to" + el valor del                                                  composito, con un step del valor del composito.
    
* cto = cfrom + cint                           "cto" es el cfrom + el valor del composito. ej: 0 + 5 = 5 = [0,5]

* clen = np.zeros (cto.shape)                  "clen" queda relleno con valores "0" en base a la cantidad de valores to.
* cvar = np.zeros (cto.shape)                  "var" = "clen"
* cvar[:] = np.nan                             Luego "cvar" se convierte en valores "NaN"
* cacum = np.zeros (cto.shape)                 "cacum" completo de valores cero.

* iprop =np.zeros(ito.shape)                    

**¿Cúal es el criterio para la longitud de compositación?**

* Moda estadística para este caso, aunque bien sabemos que hay otras.

In [31]:
df['len'].mode() # encontrando la moda de la longitud de testigos.

0    5
dtype: int64

In [32]:
len_com = df['len'].mode() # Longitud de composito es igual a 5 m.

In [33]:
d2 = {'dhid': [1,1,1,1], 'from': list(range(0,16,5)), 'to': list(range(5,21,5))}

df2 = pd.DataFrame(data=d2)

a = np.zeros(df2['to'].shape) #Rutina que crea valores cero (opción 1)

b = np.nan # Directamente se le asigna un valor perdido NaN

df2['grade'] = b


df2['len'] = (df2['to'] - df2['from'])

df2['len_acum'] = df2.len.cumsum()


df2.head() # Este df será conocido como df de sálida.

,dhid,from,to,grade,len,len_acum
0,1,0,5,NaN,5,5
1,1,5,10,NaN,5,10
2,1,10,15,NaN,5,15
3,1,15,20,NaN,5,20


**¿Cúal es el valor "grade" de los 4 compositos de 5 m?**

* Hay varias formas distintas de llegar al mismo resultado, esta vez plantare la semilla con algo muy básico:

In [34]:
df.head() #5 filas de datos

,dhid,from,to,grade,len,len_acum
0,1,0,1,1,1,1
1,1,1,3,2,2,3
2,1,3,8,3,5,8
3,1,8,15,4,7,15
4,1,15,20,5,5,20


In [35]:
df2.head() # 4 filas de datos

,dhid,from,to,grade,len,len_acum
0,1,0,5,NaN,5,5
1,1,5,10,NaN,5,10
2,1,10,15,NaN,5,15
3,1,15,20,NaN,5,20


**Matriz desarrollo:**

In [36]:
df2.len_acum.iloc[-1] # N° de filas del df.

d_com = {'dhid': 1, 'from': list(range(0,20,1)), 'to': list(range(1,21,1))} # creando las 3 primeras columnas.

df_com = pd.DataFrame(data=d_com) # creando el DataFrame.

df_com['len'] = (df_com['to'] - df_com['from']) # creando la columna de longitud de testigos.

df_com['len_acum'] = df_com.len.cumsum() # creando la columna longitud acumulada. 

df_com['grade'] = np.nan # creando la columna de leyes con valor por defecto NaN

df_com.at[:, 'grade'] = df.grade.repeat(df['len']).reset_index() # dando valor a la columna de leyes. 

com = float(input("Ingrese el valor del composito: ")) # El input de este ejemplo esta hecho solo para ingresar "5" que es la moda.

df_com.at[:, 'work'] = ((df_com['grade'] * df_com['len'])/com) # grade * len / len composito

# 20/5 = 4 # al compositar a 5 m se crean 4 leyes

a = df_com.work[:5].sum(axis=0) # selección de filas x 5 con autosuma
b = df_com.work[5:10].sum(axis=0) 
c = df_com.work[10:15].sum(axis=0) 
d = df_com.work[15:20].sum(axis=0) 

leyes = [a,b,c,d]

df2['grade'] = leyes # se pasan las leyes a la matriz B
    
print(df2.head(5))

Ingrese el valor del composito: 5
   dhid  from  to  grade  len  len_acum
0     1     0   5    2.2    5         5
1     1     5  10    3.4    5        10
2     1    10  15    4.0    5        15
3     1    15  20    5.0    5        20


**Al regularizar el tamaño del soporte de la muestra (compositación) la dispersión disminuye:**

In [40]:
print("Varianza A: ", df['grade'].var(), "Ley media A: ", df['grade'].mean())

Varianza A:  2.5 Ley media A:  3.0


In [42]:
print("Varianza B: ", round(df2['grade'].var(),2), "Ley media B: ", df2['grade'].mean())

Varianza B:  1.37 Ley media B:  3.65
